# Python calculation to be made for individual models

## 0. Packages and data

In [1]:
path='G:/Shared drives/BeStarsMiMeS/UpdatedFiles/' #Patrick's google file stream path

In [2]:
import specpolFlow as pol
import numpy as np
import pandas as pd

## 0.5 Calculation of LSD Parameters

The next block code calculates the adequate parameters for LSD calculations, given a velocity factor (`vel_fact`) for vsini. For example if vel_fact=5, then the profiles are calculated in the range [-5*vsini,5*vsini]. After the calculations, the parameters are stored in the file *'Params-LSDProfiles-CalculatedByPython.csv'* inside the LSD folder. 

After that, there are different codes for different sets of LSD profile calculations. The first cell in all of them reads the *'Params-LSDProfiles-CalculatedByPython.csv'* file to get the parameters. The next loop calculates the LSD profiles for a given **spectra** and **mask**. The calculations are saved in *'.lsd'* files in respective folders inside the LSD folder, and a pdf containing all plots is saved in a file in the LSD folder. 

We have opted to calculate the profiles in a range corresponding to $\pm3 \times v\sin i$, allowing some extra room in case of large radial velocity. The pixel size is set to 20 pixels per $v\sin i$, with a minimum of 2.6 km/s.  

> Add the extra explanation from Federico's slide deck, or add a link to it?

In [3]:
import pandas as pd

# Using pandas to read spreadsheet

sheet_id='1M6y1Wnsrc-w5FjUMfKaSFa_-foIDAaMe8W4lYNWnWyk'
sheet_name = 'Observations'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
Observations=pd.read_csv(url)

sheet_name = 'Stars'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
StarData=pd.read_csv(url)


In [4]:
vel_fac = 3
data_per_vsini = 20

In [5]:
maxv={} #finding the maximum radial velocity for each observation. 
for i, ObsInfo in Observations.iterrows():
  star = ObsInfo["Name"]
  try:
    maxv[star]
  except:
    maxv.update({star: abs(ObsInfo["Unnamed: 11"])})
  else:
    if maxv[star] < abs(ObsInfo["Unnamed: 11"]):
      maxv[star] = abs(ObsInfo["Unnamed: 11"])

data = {}
starA = []
velSA = []
velFA = []
velPA = []

In [12]:
for i, ObsInfo in Observations.iterrows():
  star = ObsInfo["Name"]
  obs = ObsInfo["Unnamed: 2"]
  for j in range(0,len(StarData["Name"])):
    if StarData["Name"][j] == star:
      modelcode = StarData["ModelCode"][j].upper()
      if StarData['vsini-estimate'][j]=='very bad' or np.isnan(StarData['vsini-estimate'][j]):
        vsini=StarData['Adopted-vsini'][j]
      else:
        vsini = float(StarData['vsini-estimate'][j])      

  if maxv[star] + vsini < vel_fac*vsini:
    velS , velF = (- vel_fac*vsini,vel_fac*vsini) 
  else:
    velS , velF = (- vel_fac*vsini - maxv[star],vel_fac*vsini + maxv[star])
  
  velP = vsini / data_per_vsini
  if velP < 2.6:
    velP = 2.6

  if star not in starA:
    starA.append(star)
    velSA.append(velS)
    velFA.append(velF)
    velPA.append(velP)

dat = {'Name': starA,
       'VelStart':velSA,
       'VelEnd':velFA,
       'VelPixel':velPA}

df = pd.DataFrame(data=dat)
df.to_csv('{}03-LSD-calculations/LSD/Params-LSDProfiles-CalculatedByPython.csv'.format(path))


## 1. Cleaning 0.02 Depth Masks

In [6]:
## masks a mask for each MODEL where the balmer exclusion region is 800+2*vsini km/s
for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
        if StarData["Name"][j] == star:
            modelcode = StarData["ModelCode"][j].upper()
            if StarData['vsini-estimate'][j]=='very bad' or np.isnan(StarData['vsini-estimate'][j]):
                vsini=StarData['Adopted-vsini'][j]
            else:
                vsini = float(StarData['vsini-estimate'][j])      

    velrange = 800+2*vsini # units are in km/s
    excluded_regions = pol.get_Balmer_regions_default(velrange) + pol.get_telluric_regions_default()

    clean_Mask_filename = '{}02-Mask-calculations/MaskCleaning/depth0.02_tellClean/{}_{}_depth0.02.mask'.format(path,modelcode,star)

    mask_filename= '{}02-Mask-calculations/VALDlist2mask/depth0.02/{}_depth0.02.mask'.format(path,modelcode)
    mask = pol.read_mask(fname=mask_filename)

    mask.clean(excluded_regions).save(clean_Mask_filename)

## 2. Cleaning Asif Masks

In [7]:
## masks a mask for each MODEL where the balmer exclusion region is 800+2*vsini km/s
for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
        if StarData["Name"][j] == star:
            modelcode = StarData["ModelCode"][j].upper()
            if StarData['vsini-estimate'][j]=='very bad' or np.isnan(StarData['vsini-estimate'][j]):
                vsini=StarData['Adopted-vsini'][j]
            else:
                vsini = float(StarData['vsini-estimate'][j])      

    velrange = 800+2*vsini # units are in km/s
    excluded_regions = pol.get_Balmer_regions_default(velrange) + pol.get_telluric_regions_default()

    clean_Mask_filename = '{}02-Mask-calculations/MaskCleaning/AsifMaskClean_tellClean/{}_clean_{}.mask'.format(path,star,obs)

    mask_filename= '{}00-InputMaterial/AsifMaskClean/{}_clean_{}.mask'.format(path,star,obs)
    mask = pol.read_mask(fname=mask_filename)
    mask.iuse[np.where(mask.depth<=0.0001)]=0

    mask.clean(excluded_regions).save(clean_Mask_filename)

In [8]:
## masks a mask for each MODEL where the balmer exclusion region is 800+2*vsini km/s
for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
        if StarData["Name"][j] == star:
            modelcode = StarData["ModelCode"][j].upper()
            if StarData['vsini-estimate'][j]=='very bad' or np.isnan(StarData['vsini-estimate'][j]):
                vsini=StarData['Adopted-vsini'][j]
            else:
                vsini = float(StarData['vsini-estimate'][j])      

    velrange = 800+2*vsini # units are in km/s
    excluded_regions = pol.get_Balmer_regions_default(velrange) + pol.get_telluric_regions_default()

    clean_Mask_filename = '{}02-Mask-calculations/MaskCleaning/AsifMaskCleanTweak_tellClean/{}_clean_{}.mask_auto_twk'.format(path,star,obs)

    mask_filename= '{}00-InputMaterial/AsifMaskCleanTweak/{}_clean_{}.mask_auto_twk'.format(path,star,obs)
    mask = pol.read_mask(fname=mask_filename)
    mask.iuse[np.where(mask.depth<=0.0001)]=0

    mask.clean(excluded_regions).save(clean_Mask_filename)

## 3. Super Cleaning Asif Masks

In [9]:
## masks a mask for each MODEL where the balmer exclusion region is 800+2*vsini km/s
for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
        if StarData["Name"][j] == star:
            modelcode = StarData["ModelCode"][j].upper()
            if StarData['vsini-estimate'][j]=='very bad' or np.isnan(StarData['vsini-estimate'][j]):
                vsini=StarData['Adopted-vsini'][j]
            else:
                vsini = float(StarData['vsini-estimate'][j])      

    velrange = 800+2*vsini # units are in km/s
    excluded_regions = pol.get_Balmer_regions_default(velrange) + pol.get_telluric_regions_default()

    clean_Mask_filename = '{}02-Mask-calculations/MaskCleaning/AsifMaskSuperClean_tellClean/{}_clean_{}.mask'.format(path,star,obs)

    mask_filename= '{}00-InputMaterial/AsifMaskClean/{}_clean_{}.mask'.format(path,star,obs)
    mask = pol.read_mask(fname=mask_filename)
    mask.iuse[np.where(mask.depth<0.02)]=0

    mask.clean(excluded_regions).save(clean_Mask_filename)

In [10]:
## masks a mask for each MODEL where the balmer exclusion region is 800+2*vsini km/s
for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
        if StarData["Name"][j] == star:
            modelcode = StarData["ModelCode"][j].upper()
            if StarData['vsini-estimate'][j]=='very bad' or np.isnan(StarData['vsini-estimate'][j]):
                vsini=StarData['Adopted-vsini'][j]
            else:
                vsini = float(StarData['vsini-estimate'][j])      

    velrange = 800+2*vsini # units are in km/s
    excluded_regions = pol.get_Balmer_regions_default(velrange) + pol.get_telluric_regions_default()

    clean_Mask_filename = '{}02-Mask-calculations/MaskCleaning/AsifMaskSuperCleanTweak_tellClean/{}_clean_{}.mask_auto_twk'.format(path,star,obs)

    mask_filename= '{}00-InputMaterial/AsifMaskCleanTweak/{}_clean_{}.mask_auto_twk'.format(path,star,obs)
    mask = pol.read_mask(fname=mask_filename)
    mask.iuse[np.where(mask.depth<0.02)]=0

    mask.clean(excluded_regions).save(clean_Mask_filename)

In [11]:
pd.DataFrame(excluded_regions.to_dict())

,start,stop,type
0,653.478929,659.083071,Halpha
1,484.064367,488.215633,Hbeta
2,432.196771,435.903229,Hgamma
3,408.418730,411.921270,Hdelta
4,395.314918,398.705082,Hepsilon
5,360.000000,392.000000,Hjump
6,587.500000,592.000000,telluric
7,627.500000,632.500000,telluric
8,684.000000,705.300000,telluric
9,717.000000,735.000000,telluric


## OLD

### 1 Remove lines in the mask that are in regions of heavy telluric contamination - Federico

### Based on the _04-Telluric contamination_ notebook:

In the _04-Telluric contamination_ notebook, a code was written to use only spectral lines which are not in the contaminated regions of the spectrum. These regions were identified visually and put in the spreadsheet _04-ContaminationregionsWL_. 

In this notebook we create a loop over all the individual models, with different mask values to 'clean' them.


The two next code blocks just makes sure we are located correctly and authenticates the user. However we change the directory to the _/Masks_ folder. 

The next line creates a new folder called __depth0.02_tellClean__, where the new masks will be stored. 

In [ ]:
#!mkdir depth0.02_tellClean

The next code block gathers information from the _00InputInformation_ and _04-ContaminationregionsWL_ spreadsheets using pandas. 

In [14]:
import pandas as pd

# Using pandas to read spreadsheet

sheet_id='1M6y1Wnsrc-w5FjUMfKaSFa_-foIDAaMe8W4lYNWnWyk'
sheet_name = 'ListOfModels'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
lom=pd.read_csv(url)


#Regions

sheet_id='19lS0Xg-2ZUs0ps8jZ-JM3pR1YIuC_lWvRMpFAM5VUYI'
sheet_name = 'WLRegions'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
WLRegions=pd.read_csv(url)


In [15]:
WLRegions

,WLStart,WLFinish,Type
0,587.500000,592.000000,Telluric
1,627.500000,632.500000,Telluric
2,684.000000,705.300000,Telluric
3,717.000000,735.000000,Telluric
4,757.000000,771.000000,Telluric
5,790.000000,794.000000,Telluric
6,809.000000,990.000000,Telluric
7,360.000000,392.000000,Hjump
8,654.112333,658.487667,Halpha
9,484.519533,487.760467,Hbeta


This last code block uses the code from _04-Telluric contamination_ notebook, but loops for all model masks and finally saves the masks with the same name, but located in a the new folder that was created previously. 

In [ ]:
#note: this function is now included within specpolFlow
def clean_model_mask(name_in, name_out):
  '''Function that cleans the mask by model code'''
  #Place the regions into a matrix
  L = np.shape(WLRegions.loc[:,'WLStart'])[0]

  regions = np.zeros((L,2))

  regions[:,0] = WLRegions.loc[:,'WLStart']
  regions[:,1] = WLRegions.loc[:,'WLFinish']

  #looping over all the model codes
  for i in range(0,len(lom)): 
  #for i in range(1):
    modelcode =lom["Model Code"][i].upper()
    
    #The next loop loops over every observation, which is needed
    #for Asif's masks, so better leave it this way to change
    #easier between masks although our masks only depende on the
    #modelcode

    name = '{}02-Mask-calculations/VALDlist2mask/'+name_in.format(path,modelcode)
    mask_i = pol.read_mask(fname=name)

    #Identifying if a line is inside or outside the regions
    dcard = []
    for lines in mask_i.wl:
      for i in range(0,np.size(regions[:,0])):
        if regions[i,0] < lines < regions[i,1]:
          dcard.append(lines)

    #Creating a list with the mask.wl elements
    mask_list = mask_i.wl.tolist()

    for line in dcard:
      index = (mask_list).index(line)
      mask_i.iuse[index]=0

    for j in range(mask_i.wl.size):
      if mask_i.wl[j]==370.5001 or mask_i.wl[j]==381.9603 or mask_i.wl[j]==387.9772:
        mask_i.iuse[j]=0

    l = np.size(mask_i.wl)
    spec_lines = []
    for i in range(0,l):
      if mask_i.iuse[i]==1:
        spec_lines.append(mask_i.wl[i])
    mask_i.save('{}02-Mask-calculations/MaskCleaning/'+name_out.format(path,modelcode))
  return()


In [ ]:
clean_model_mask('depth0.1/{}_depth0.1.mask', 'depth0.1_tellClean/{}_depth0.1.mask')

clean_model_mask('depth0.02/{}_depth0.02.mask', 'depth0.02_tellClean/{}_depth0.02.mask')

### 2. Remove the lines in Asif's and Coralie's masks that are in regions of contamination (and also fix some lines with zero depth but in use in the tweaked masks)

Their masks have tailored masks for each individual observations. So we loop over all of these below. 

Furthermore, some of the tweaked masks have lines that are used, but have the depth set to zero. We remove these lines (by setting the last column to zero). 

In [111]:
import pandas as pd

# Using pandas to read spreadsheet

sheet_id='1M6y1Wnsrc-w5FjUMfKaSFa_-foIDAaMe8W4lYNWnWyk'
sheet_name = 'Observations'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
obs=pd.read_csv(url)


#Regions

sheet_id='19lS0Xg-2ZUs0ps8jZ-JM3pR1YIuC_lWvRMpFAM5VUYI'
sheet_name = 'WLRegions'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
WLRegions=pd.read_csv(url)


In [108]:
#note: this function is now included within specpolFlow
def clean_obs_mask(name_in, name_out):
  '''Function to clean the mask by observation'''

  #Place the regions into a matrix
  L = np.shape(WLRegions.loc[:,'WLStart'])[0]

  regions = np.zeros((L,2))

  regions[:,0] = WLRegions.loc[:,'WLStart']
  regions[:,1] = WLRegions.loc[:,'WLFinish']

  #Getting masks
  for i in range(0,len(obs)): 
    star = obs["Name"][i]
    o = obs["Unnamed: 2"][i]
    
    #The next loop loops over every observation
    name = '{}00-InputMaterial/'.format(path)+name_in.format(star,o)
    mask_i = pol.read_mask(fname=name)

    #Identifying if a line is inside or outside the regions
    dcard = []
    for lines in mask_i.wl:
      for i in range(0,np.size(regions[:,0])):
        if regions[i,0] < lines < regions[i,1]:
          dcard.append(lines)

    #Creating a list with the mask.wl elements
    mask_list = mask_i.wl.tolist()

    for line in dcard:
      index = (mask_list).index(line)
      mask_i.iuse[index]=0

    l = np.size(mask_i.wl)
    spec_lines = []
    for i in range(0,l):
      if mask_i.iuse[i]==1:
        spec_lines.append(mask_i.wl[i])

    ## Check for lines that have zero depth
    # (the depths have 3 decimal places)
    mask_i.iuse[mask_i.depth <= 0.0001] = 0.0

    mask_i.save('{}02-Mask-calculations/MaskCleaning/'.format(path)+name_out.format(star,o))

  return

In [34]:
clean_obs_mask('AsifMaskClean/{}_clean_{}.mask', 'AsifMaskClean_tellClean/{}_clean_{}.mask')

clean_obs_mask('AsifMaskCleanTweak/{}_clean_{}.mask_auto_twk', 'AsifMaskCleanTweak_tellClean/{}_clean_{}.mask_auto_twk')


In [114]:
#THIS FUNCTION SETS THE DEPTH CUTOFF TO 0.02. WHICH IS EQUAL TO OUR FULL MASK

def super_clean_obs_mask(name_in, name_out):
  '''Function to clean the mask by observation'''

  #Place the regions into a matrix
  L = np.shape(WLRegions.loc[:,'WLStart'])[0]

  regions = np.zeros((L,2))

  regions[:,0] = WLRegions.loc[:,'WLStart']
  regions[:,1] = WLRegions.loc[:,'WLFinish']

  #regions[:,0]=pd.DataFrame(excluded_regions.to_dict())['start']
  #regions[:,1]=pd.DataFrame(excluded_regions.to_dict())['stop']

  #Getting masks
  for i in range(0,len(obs)): 
    star = obs["Name"][i]
    o = obs["Unnamed: 2"][i]
    
    #The next loop loops over every observation
    name = '{}00-InputMaterial/'.format(path)+name_in.format(star,o)
    mask_i = pol.read_mask(fname=name)

    #Identifying if a line is inside or outside the regions
    dcard = []
    for lines in mask_i.wl:
      for i in range(0,np.size(regions[:,0])):
        if regions[i,0] < lines < regions[i,1]:
          dcard.append(lines)

    #Creating a list with the mask.wl elements
    mask_list = mask_i.wl.tolist()

    for line in dcard:
      index = (mask_list).index(line)
      mask_i.iuse[index]=0

    l = np.size(mask_i.wl)
    spec_lines = []
    for i in range(0,l):
      if mask_i.iuse[i]==1:
        spec_lines.append(mask_i.wl[i])

    ## Check for lines that have zero depth
    # (the depths have 3 decimal places)
    mask_i.iuse[mask_i.depth <= 0.02] = 0.0

    mask_i.save('{}02-Mask-calculations/MaskCleaning/'.format(path)+name_out.format(star,o))

  return

In [115]:
super_clean_obs_mask('AsifMaskClean/{}_clean_{}.mask', 'AsifMaskSuperClean_tellClean/{}_clean_{}.mask')

#super_clean_obs_mask('AsifMaskCleanTweak/{}_clean_{}.mask_auto_twk', 'AsifMaskSuperCleanTweak_tellClean/{}_clean_{}.mask_auto_twk')

### 3. Extended H lines

In [7]:
##Retrieving Calculated LSD Parameters
df = pd.read_csv('{}03-LSD-calculations/LSD/Params-LSDProfiles-CalculatedByPython.csv'.format(path))

blend=np.array(df['Name'][df['VelEnd']>=900])
blend


array(['hd20336', 'hd33328', 'hd45725', 'hd58978', 'hd77320', 'hd89884',
       'hd138749', 'hd164284', 'hd164906', 'hd191610', 'hd217050'],
      dtype=object)

In [8]:
import astropy.units as u
import astropy.constants as const
import numpy as np

In [13]:
Halpha=656.3
Hbeta=486.14
Hgamma=434.05
Hdelta=410.17
Hepsilon=397.01

#inputs.
lines=np.array([Halpha,Hbeta,Hgamma,Hdelta,Hepsilon])*u.nm
Hjump=np.array([360,392])*u.nm
tellstart=np.array([587.5,627.5,684.0,717.0,757.0,790.0,809.0])#*u.nm
tellend=np.array([592.0,632.5,705.3,735.0,771.0,795.0,990.0])#*u.nm


def exclude_regions(lines,Hjump,velrange,tellstart,tellend):
    jumpend=(velrange/const.c*Hjump+Hjump).to(u.nm)/u.nm
    linestart=(-velrange/const.c*lines+lines).to(u.nm)/u.nm
    lineend=(velrange/const.c*lines+lines).to(u.nm)/u.nm
    linestart=np.append(linestart,jumpend[0])
    lineend=np.append(lineend,jumpend[1])

    data=pd.DataFrame(data={"WLStart": np.concatenate((tellstart,linestart)), 'WLFinish': np.concatenate((tellend,lineend))})
    return data
    
for i in range(len(blend)):
    hd=blend[i]
    starstart=np.array(df['VelStart'][df['Name']==hd])[0]
    starend=np.array(df['VelEnd'][df['Name']==hd])[0]

    lom=np.array(StarData['ModelCode'][StarData['Name']==hd])[0]

    velrange=(starend+700)*u.km/u.s
    data=exclude_regions(lines,Hjump,velrange,tellstart,tellend)

    def clean_model_mask(name_in, name_out):
        '''Function that cleans the mask by model code'''
        #Place the regions into a matrix
        L = np.shape(data.loc[:,'WLStart'])[0]

        regions = np.zeros((L,2))

        regions[:,0] = data.loc[:,'WLStart']
        regions[:,1] = data.loc[:,'WLFinish']

        #looping over all the model codes
        #for i in range(0,len(lom)): 
        for i in range(1):
            modelcode =lom.upper()
    
        #The next loop loops over every observation, which is needed
        #for Asif's masks, so better leave it this way to change
        #easier between masks although our masks only depende on the
        #modelcode

        name = '{}02-Mask-calculations/VALDlist2mask/'.format(path)+name_in.format(modelcode)
        mask_i = pol.read_mask(fname=name)

        #Identifying if a line is inside or outside the regions
        dcard = []
        for lines in mask_i.wl:
            for j in range(0,np.size(regions[:,0])):
                if regions[j,0] < lines < regions[j,1]:
                    dcard.append(lines)

        #Creating a list with the mask.wl elements
        mask_list = mask_i.wl.tolist()

        for line in dcard:
            index = (mask_list).index(line)
            mask_i.iuse[index]=0

        for j in range(mask_i.wl.size):
            if mask_i.wl[j]==370.5001 or mask_i.wl[j]==381.9603 or mask_i.wl[j]==387.9772:
                mask_i.iuse[j]=0
    
        l = np.size(mask_i.wl)
        spec_lines = []
        for j in range(0,l):
            if mask_i.iuse[j]==1:
                spec_lines.append(mask_i.wl[j])
        mask_i.save('{}02-Mask-calculations/MaskCleaning/'.format(path)+name_out.format(hd))
        return()
    clean_model_mask('depth0.1/{}_depth0.1.mask', '{}_depth0.1.mask')
    clean_model_mask('depth0.02/{}_depth0.02.mask', '{}_depth0.02.mask')